## Importing required libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pprint import pprint

## Reading iris dataset into dataframe

removing Id column as that isnt a feature. Changing column names to improve readability

In [2]:
df = pd.read_csv('iris.csv')
df.drop('Id', axis = 1, inplace= True)
cols = list(df.columns)
df = df.rename(columns={cols[0]: "Sepal Length", cols[1]: "Sepal Width", cols[2]: "Petal Length", cols[3]: "Petal Width", cols[4]: "Label"})

In [3]:
df.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
data = df.values # a 2D numpy array corresponding to the dataset (without column headers)
targets = np.unique(data[:,-1]) # a list of all output classes or targets or labels for iris flowers in the dataset
features = list(df.columns) # a list of all features in the dataset. Also contains 'Label', which is not a feature

In [5]:
# x = data[:,0:4]
# y = data[:,-1]

# print("data shapes:",x.shape, y.shape)
# x_train, x_test, y_train, y_test = train_test_split(x,y, random_state = 0)
# print("train data shapes:",x_train.shape, y_train.shape,"\ntest data shapes",x_test.shape,y_test.shape)

## Functions used to implement the decision tree

In [5]:
def check_purity(data):
    ''' Is used to check if the data belonging to a node is pure, i.e. if all flowers contained in the node are of 
    the same type. If the node is pure, it returns true otherwise false.'''
    label_column = data[:, -1] # np array containing labels for each data point
    unique_classes = np.unique(label_column) # np array containing all unique labels in the node
    #same as np.array(list(set(label_column)))

    if len(unique_classes) == 1: #pure node ie leaf reached
        return True
    else:
        return False

In [6]:
def classify_data(data):
    ''' Returns the output class for the node by majority vote.'''
    label_column = data[:, -1] # np array containing labels for each data point
    unique_classes, class_counts = np.unique(label_column, return_counts=True) #returns 2 lists. 
    #unique classes contains all unique labels contained in the node 
    #class_counts is an array containing the number of times each unique label appears in the data,
    #in the same order as in unique_classes

    index = class_counts.argmax() #index of element having highest value in class_counts
    output_class = unique_classes[index] #contains label corresponding to index ie. output class that appears most 
    #frequently in the data
    
    return output_class

In [7]:
def get_potential_splits(data):
    '''Returns a dictionary poetntial_splits, containing index of the feature/column as the key with a list as
    its value, that contains all potential splits for that feature.'''
    potential_splits = {} #creates empty dictionary
    _, n_columns = data.shape #n_columns corresponds to number of columns in the data
    for column_index in range(n_columns - 1):   # excluding the last column as it is the label
        potential_splits[column_index] = []     # creating empty list as value for column index key
        values = data[:, column_index]          # values contains all values in the current column in the data
        unique_values = np.unique(values)       # all unique values are found among them

        for index in range(1,len(unique_values)): 
            current_value = unique_values[index]
            previous_value = unique_values[index - 1]
            potential_split = (current_value + previous_value) / 2    #potential split is the mid point
                                                                      #between every 2 consecutive (unique) values
            potential_splits[column_index].append(potential_split) #append potential split found to value list 
    
    return potential_splits

In [8]:
def split_data(data, split_column, split_value):
    '''Splits the data passed to it about the split value of the split column.'''
    split_column_values = data[:, split_column]        # array containing all values of the split_column in the data
    data_split = np.delete(data,split_column,axis = 1) #removing split_column from the data as that feature is not 
                                                       #to be used again to split
    data_below = data_split[split_column_values <= split_value] #all datapoints having value of split_column <=
                                                                #the split_value
    data_above = data_split[split_column_values >  split_value] #all datapoints having value of split_column > 
                                                                #the supplied split_value
    
    return data_below, data_above                               #split data is returned

In [9]:
def calculate_entropy(data):
    '''Used to calculate entropy of the data.'''
    label_column = data[:, -1]                              #labels in the data
    _, counts = np.unique(label_column, return_counts=True) #only counts returned by unique are used

    probabilities = counts / counts.sum()                   #as numpy arrays perform element-wise operations, 
                                                            #probabilities are found as count of the label/total count of all labels
    entropy = sum(probabilities * -np.log2(probabilities)) #formula for entropy summed over all unqiue classes
     
    return entropy

In [10]:
def calculate_overall_entropy(data_below, data_above):
    '''Used to find entropy after a split is performed.'''
    n = len(data_below) + len(data_above)                            #total number of data points
    p_data_below = len(data_below) / n                               #probabilities are found for splits
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below)  #weighted addition of required information or 
                      + p_data_above * calculate_entropy(data_above)) #entropies
    
    return overall_entropy

In [11]:
def determine_best_split(data, potential_splits):
    '''Is used to determine the best possible split (having highest information gain) out of all potential splits.'''
    overall_entropy = 9999                           #an arbitrary high value
    for column_index in potential_splits:            #iterate over column indices in potential_splits
        for value in potential_splits[column_index]: #iterate over all potential splits for that column
            data_below, data_above = split_data(data, split_column=column_index, split_value=value) #perform split
            current_overall_entropy = calculate_overall_entropy(data_below, data_above) #find entropy after split
            if current_overall_entropy <= overall_entropy: #split resulting in lowest entropy after split is chosen
                overall_entropy = current_overall_entropy  #as that will lead to max information gain
                best_split_column = column_index           #and thus will be the best possible split
                best_split_value = value
    #best split column, value and resulting entropy is returned
    return best_split_column, best_split_value, overall_entropy

In [12]:
def build_decision_tree(data, features, counter=0):
    '''Recursive function to construct decision tree. The tree is a dictionary containing questions
    (corresponding to the split that is decided) as keys with key values being a list of 2 elements corresponding to 
    if the answer to the question is yes and no respectively.'''
    # base cases: if data is pure or all features have been used up to make splits
    if (check_purity(data)) or (len(features)==0):
        classification = classify_data(data)
        return classification #majority vote 

    
    # recursive part
    else:    
        counter += 1
        # calling helper functions 
        potential_splits = get_potential_splits(data) # a dictionary of potential splits
        split_column, split_value, _ = determine_best_split(data, potential_splits) #_ because overall entropy is not used 
        feature_name = features[split_column] #feature name is found using column index 
        features.remove(feature_name) #split feature is removed from list of features so no more splits are made on
                                      #the same feature
        data_below, data_above = split_data(data, split_column, split_value) #data is split according to the
                                                                             #best split that was found 
        
       
        # instantiate sub-tree
        question = "{} <= {}".format(feature_name, split_value) #split recorded in sub tree as a question
        sub_tree = {question: []} #empty list created as value for the key that is the question
        
        # find answers (recursion)
        yes_answer = build_decision_tree(data_below, features, counter) #recursive call for 'yes' answer of question
        no_answer = build_decision_tree(data_above, features, counter) #recursive call for 'no' answer of question
        
        sub_tree[question].append(yes_answer) #append answers to list for question
        sub_tree[question].append(no_answer)
        
        return sub_tree

In [13]:
def display_output(tree, data, features, counter = 0):
    '''Uses the tree created to print output in the required format.'''
    counter += 1 #increment counter to keep track of levels
    label_column = data[:,-1] #contains all labels in the data
    targets, counts = np.unique(label_column, return_counts=True) # 2 lists containing all unique targets or labels
                                                                  # and their counts respectively
    print("Level", counter-1)
    for i in range(len(counts)):
        print("Count of", targets[i] ,"=",counts[i])
    current_entropy = calculate_entropy(data)                     # to find current entropy
    print("Current entropy is =", current_entropy)
    if current_entropy == 0:                                      # base case: leaf node is reached
        print("Reached leaf node\n")
    
    elif len(features)==0:                                        # base case: when all features have been used up
        print("No more features left to split upon")
        return
    
    else:                                                         # recursion will take place here
        key = list(tree.keys())[0]                                # to find the key
        question = key.split()                                    # list of words and operators in the question 
        feature_name = " ".join(question[:2]) #feature name is obtained by joining first two words of the question
        split_value = float(question[-1])     #split value is the last term in the question. it has to be converted 
                                              #to float to make comparisions later
        print("Splitting on feature", feature_name,"with gain ratio ")
        index = features.index(feature_name)  #to find index of splitting feature in features list
        data_below, data_above = split_data(data, index, split_value) #perform split
        split_entropy = calculate_overall_entropy(data_below, data_above) #find entropy
        features.remove(feature_name) #remove feature that was used to make split from list of features
        
        info_gain = current_entropy - split_entropy #find information gain
        r = counts / counts.sum()
        split_info = sum(-r * np.log10(r)) #find split info
        gain_ratio = info_gain/split_info  #find gain ratio
        print(gain_ratio) 
        print()
    
        #recursive calls
        left_child = tree[key][0] #find left child of current node in tree
        display_output(left_child, data_below, features, counter) #call on left child
        right_child = tree[key][1] #find right child of current node in tree
        display_output(right_child, data_above, features, counter) #call on right child


## Decision Tree

The decision tree has been implemented as a dictionary with questions corresponding to the decided split as the key, having 2 values in a list. The first value corresponds to if the answer to the question is yes and the second to if the answer is no.   
The answers may be a string, containing the output class, in case a pure or leaf node has been reached or if all features have been used up and no more splits can be performed.   
Otherwise, the answer is a dictionary of the same format, corresponding to a subtree characterising further splits.

In [14]:
tree = build_decision_tree(data,features[:4]) #features slice is passed so as to not pass the string 'Label' as
print(tree,"\n")                              #it is not a feature name. print the tree here. 
# pprint(tree)

{'Petal Width <= 0.8': ['Iris-setosa', {'Petal Length <= 4.75': [{'Sepal Length <= 4.95': [{'Sepal Width <= 2.45': ['Iris-versicolor', 'Iris-virginica']}, 'Iris-versicolor']}, 'Iris-virginica']}]} 



## Output
Here is the tree printed in the required format.

In [16]:
display_output(tree, df.values, features[:4]) #output

Level 0
Count of Iris-setosa = 50
Count of Iris-versicolor = 50
Count of Iris-virginica = 50
Current entropy is = 1.584962500721156
Splitting on feature Petal Width with gain ratio 
1.924659245361106

Level 1
Count of Iris-setosa = 50
Current entropy is = 0.0
Reached leaf node

Level 1
Count of Iris-versicolor = 50
Count of Iris-virginica = 50
Current entropy is = 1.0
Splitting on feature Petal Length with gain ratio 
2.1837483292096787

Level 2
Count of Iris-versicolor = 44
Count of Iris-virginica = 1
Current entropy is = 0.15374218032876188
Splitting on feature Sepal Length with gain ratio 
2.3616109695158807

Level 3
Count of Iris-versicolor = 1
Count of Iris-virginica = 1
Current entropy is = 1.0
Splitting on feature Sepal Width with gain ratio 
3.321928094887362

Level 4
Count of Iris-versicolor = 1
Current entropy is = 0.0
Reached leaf node

Level 4
Count of Iris-virginica = 1
Current entropy is = 0.0
Reached leaf node

Level 3
Count of Iris-versicolor = 43
Current entropy is = 0